In [1]:
# Autoreloading
%load_ext autoreload
%autoreload 2

In [1]:
# Imports
from cluster_client.manager import ClusterManager
from cluster_client.model.field_expressions import *
from cluster_client.model.aggregate_expressions import *
from cluster_client.upload.cassandra import *
from cluster_client.connector.cassandra import CassandraConnector
from cluster_client.test_data.generate_test_loan_data import loan_amortisation_generator, loan_amortisation_metadata, loan_origination_generator, loan_origination_metadata
from tqdm import tqdm

In [ ]:
# Upload
connector = CassandraConnector("localhost", 9042)
upload_handler = CassandraUploadHandler(connector)
upload_handler.create_from_csv(r"D:\uni\Y4S1\Project\forex.csv", "test", "forex_table", ["slug"], ["date"])

In [ ]:
# Upload from iterator
connector = CassandraConnector("localhost", 9042)
upload_handler = CassandraUploadHandler(connector)
(column_names, column_types) = loan_origination_metadata()
upload_handler.drop_table("origination", "data_1000").create_from_iterator(tqdm(loan_origination_generator(1000)), "origination", "data_1000", column_names, column_types, ["loan_id"])

In [3]:
# Setup cluster manager
CLUSTER_ADDRESS = "localhost"
manager = ClusterManager(CLUSTER_ADDRESS)

In [4]:
select = manager.cassandra_table("test", "forex_table").filter(F("slug") == V("GBP/INR")).evaluate()

In [20]:
group_by = manager.cassandra_table("test", "forex_table").group_by([F("slug")], [Avg(F("high"))]).evaluate()

In [ ]:
result_df = group_by.get_dataframe()
print(result_df)